# Part 2B4 Model Training & Evaluation

In [1]:
import polars as pl
import numpy as np
import pandas as pd
import json
import pickle
import os
import gc
import psutil
import warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
from collections import defaultdict
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import joblib

warnings.filterwarnings('ignore')

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## CONFIGURATION AND SETUP

In [3]:
class FixedConfig:
    """Fixed configuration with robust parameters"""

    def __init__(self):
        # Paths
        self.OUTPUT_PATH = "/content/drive/MyDrive/Colab Notebooks/CML/Assignment 1/content/otto-output"

        # Memory management - conservative for stability
        total_memory_gb = psutil.virtual_memory().total / (1024**3)
        self.MAX_MEMORY_GB = min(40.0, total_memory_gb * 0.75)
        self.MEMORY_THRESHOLD = self.MAX_MEMORY_GB * 0.85

        # Training parameters - balanced for performance and stability
        self.TRAINING_SAMPLE_SIZE = 100000  # Reduced for stability
        self.VALIDATION_SAMPLE_SIZE = 50000  # Balanced validation size
        self.CHUNK_SIZE = 25000  # Smaller chunks for memory safety

        # LightGBM parameters - optimized and tested
        self.N_ESTIMATORS = 150  # Good performance without overfitting
        self.NUM_LEAVES = 63  # Power of 2 minus 1 for efficiency
        self.LEARNING_RATE = 0.1  # Standard learning rate
        self.MAX_DEPTH = 7  # Limited depth for regularization
        self.MIN_CHILD_SAMPLES = 50  # Reasonable regularization
        self.SUBSAMPLE = 0.8  # Feature bagging
        self.COLSAMPLE_BYTREE = 0.8  # Column sampling
        self.REG_ALPHA = 0.05  # L1 regularization
        self.REG_LAMBDA = 0.05  # L2 regularization
        self.EARLY_STOPPING_ROUNDS = 20  # Early stopping

        # GPU configuration with fallback
        self.USE_GPU = True
        self.GPU_DEVICE_ID = 0

        # Data types for memory optimization
        self.FLOAT_DTYPE = 'float32'
        self.INT_DTYPE = 'int32'

        # Evaluation parameters
        self.EVAL_AT_K = [5, 10, 20]  # Multiple evaluation points
        self.MIN_RECALL_THRESHOLD = 0.001  # Minimum acceptable performance

config = FixedConfig()

def get_memory_usage():
    """Get current memory usage in GB"""
    return psutil.virtual_memory().used / (1024**3)

def force_garbage_collection():
    """Force aggressive garbage collection"""
    for _ in range(3):
        gc.collect()

print(f"Fixed Part 2B4 Configuration:")
print(f"  Max Memory: {config.MAX_MEMORY_GB:.1f} GB")
print(f"  Training Samples: {config.TRAINING_SAMPLE_SIZE:,}")
print(f"  GPU Enabled: {config.USE_GPU}")
print(f"  Initial Memory: {get_memory_usage():.2f} GB")

Fixed Part 2B4 Configuration:
  Max Memory: 38.2 GB
  Training Samples: 100,000
  GPU Enabled: True
  Initial Memory: 1.40 GB


## ENHANCED LOGGING SYSTEM

In [4]:
def setup_enhanced_logging():
    """Setup comprehensive logging with memory tracking"""
    log_file = f"{config.OUTPUT_PATH}/fixed_model_training_log.txt"

    os.makedirs(config.OUTPUT_PATH, exist_ok=True)

    def log_message(message, level="INFO"):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        memory_gb = get_memory_usage()
        memory_pct = (memory_gb / config.MAX_MEMORY_GB) * 100
        memory_status = "OK" if memory_gb < config.MEMORY_THRESHOLD else "HIGH"

        log_entry = f"[{timestamp}] [{level}] [MEM: {memory_gb:.1f}GB/{memory_pct:.1f}%/{memory_status}] {message}"
        print(log_entry)

        try:
            with open(log_file, "a", encoding='utf-8') as f:
                f.write(log_entry + "\n")
        except:
            pass  # Continue if logging fails

        # Auto-cleanup if memory is high
        if memory_gb > config.MEMORY_THRESHOLD:
            force_garbage_collection()

    return log_message

log = setup_enhanced_logging()

log("="*80)
log("OTTO PART 2B4: FIXED MODEL TRAINING & EVALUATION STARTED")
log("="*80)

[2025-08-08 10:26:42] [INFO] [MEM: 1.4GB/3.7%/OK] ================================================================================
[2025-08-08 10:26:43] [INFO] [MEM: 1.5GB/3.8%/OK] OTTO PART 2B4: FIXED MODEL TRAINING & EVALUATION STARTED
[2025-08-08 10:26:43] [INFO] [MEM: 1.5GB/3.8%/OK] ================================================================================


## INPUT VALIDATION AND LOADING

In [5]:
def validate_and_load_inputs():
    """Enhanced input validation and loading with error handling"""
    log("Validating and loading inputs...")

    validation_results = {
        "inputs_found": {},
        "data_shapes": {},
        "feature_info": {},
        "memory_usage": get_memory_usage()
    }

    try:
        # 1. Load validation data with features
        val_data_path = f"{config.OUTPUT_PATH}/val_data_features.parquet"
        if not os.path.exists(val_data_path):
            raise FileNotFoundError(f"Required file not found: {val_data_path}")

        log("  Loading validation data with features...")
        val_data = pl.read_parquet(val_data_path)
        validation_results["inputs_found"]["val_data_features"] = True
        validation_results["data_shapes"]["val_data"] = val_data.shape
        log(f"    Loaded: {val_data.shape[0]:,} samples, {val_data.shape[1]:,} columns")

        # 2. Load feature columns
        feature_columns_path = f"{config.OUTPUT_PATH}/feature_columns.json"
        if os.path.exists(feature_columns_path):
            with open(feature_columns_path, "r") as f:
                feature_columns = json.load(f)
            validation_results["inputs_found"]["feature_columns"] = True
            validation_results["feature_info"]["available_features"] = len(feature_columns)
            log(f"    Feature columns loaded: {len(feature_columns)} features")
        else:
            # Fallback: identify feature columns automatically
            basic_columns = ["session", "aid", "type", "label"]
            feature_columns = [col for col in val_data.columns if col not in basic_columns]
            validation_results["inputs_found"]["feature_columns"] = False
            validation_results["feature_info"]["available_features"] = len(feature_columns)
            log(f"    Feature columns auto-detected: {len(feature_columns)} features", "WARN")

        # 3. Load training statistics (optional)
        stats_path = f"{config.OUTPUT_PATH}/training_data_statistics.pkl"
        training_stats = {}
        if os.path.exists(stats_path):
            try:
                with open(stats_path, "rb") as f:
                    training_stats = pickle.load(f)
                validation_results["inputs_found"]["training_statistics"] = True
                log("    Training statistics loaded")
            except:
                validation_results["inputs_found"]["training_statistics"] = False
                log("    Training statistics load failed", "WARN")
        else:
            validation_results["inputs_found"]["training_statistics"] = False
            log("    Training statistics not found", "WARN")

        # 4. Validate data quality
        required_columns = ["session", "aid", "type", "label"]
        missing_columns = [col for col in required_columns if col not in val_data.columns]

        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

        # Check for labels
        unique_labels = val_data.select("label").unique().to_series().to_list()
        if 0 not in unique_labels or 1 not in unique_labels:
            log("    Warning: Labels may not be properly set", "WARN")

        validation_results["feature_info"]["feature_columns"] = feature_columns
        validation_results["quality_checks"] = {
            "required_columns_present": len(missing_columns) == 0,
            "has_labels": len(unique_labels) >= 2,
            "sufficient_data": len(val_data) > 1000,
            "sufficient_features": len(feature_columns) >= 5
        }

        log(f"  Input validation completed successfully")
        log(f"    Memory after loading: {get_memory_usage():.2f} GB")

        return val_data, feature_columns, training_stats, validation_results

    except Exception as e:
        log(f"Input validation failed: {str(e)}", "ERROR")
        raise e

# Load inputs
val_data, feature_columns, training_stats, validation_results = validate_and_load_inputs()

[2025-08-08 10:26:43] [INFO] [MEM: 1.5GB/3.8%/OK] Validating and loading inputs...
[2025-08-08 10:26:43] [INFO] [MEM: 1.5GB/3.8%/OK]   Loading validation data with features...
[2025-08-08 10:26:50] [INFO] [MEM: 4.8GB/12.6%/OK]     Loaded: 6,617,259 samples, 67 columns
[2025-08-08 10:26:50] [INFO] [MEM: 4.8GB/12.6%/OK]     Feature columns loaded: 61 features
[2025-08-08 10:26:50] [INFO] [MEM: 4.8GB/12.6%/OK]     Training statistics loaded
[2025-08-08 10:26:51] [INFO] [MEM: 4.8GB/12.6%/OK]   Input validation completed successfully
[2025-08-08 10:26:51] [INFO] [MEM: 4.8GB/12.6%/OK]     Memory after loading: 4.82 GB


## ENHANCED DATA PREPARATION

In [6]:
class FixedDataPreprocessor:
    """Fixed data preprocessing with robust Polars to NumPy conversion"""

    def __init__(self, feature_columns: List[str]):
        self.feature_columns = feature_columns
        self.preprocessing_stats = {}

    def safe_polars_to_numpy(self, df: pl.DataFrame, columns: List[str]) -> np.ndarray:
        """Safely convert Polars DataFrame to NumPy array with multiple fallback methods"""
        try:
            # Method 1: Direct to_numpy() - simplest approach
            log(f"      Attempting direct Polars to NumPy conversion...")
            return df.select(columns).to_numpy()
        except Exception as e1:
            log(f"      Direct conversion failed: {e1}", "WARN")

            try:
                # Method 2: Convert via pandas
                log(f"      Attempting conversion via pandas...")
                pandas_df = df.select(columns).to_pandas()
                return pandas_df.values.astype(config.FLOAT_DTYPE)
            except Exception as e2:
                log(f"      Pandas conversion failed: {e2}", "WARN")

                try:
                    # Method 3: Manual column-by-column conversion
                    log(f"      Attempting manual column conversion...")
                    arrays = []
                    for col in columns:
                        col_array = df.select(col).to_series().to_numpy()
                        arrays.append(col_array)
                    return np.column_stack(arrays).astype(config.FLOAT_DTYPE)
                except Exception as e3:
                    log(f"      Manual conversion failed: {e3}", "ERROR")
                    raise ValueError(f"All conversion methods failed: {e1}, {e2}, {e3}")

    def prepare_training_data(self, data: pl.DataFrame, event_type: str,
                            sample_size: int = None) -> Tuple[np.ndarray, np.ndarray, List[str]]:
        """Prepare optimized training data for a specific event type with fixed conversion"""
        log(f"    Preparing {event_type} training data...")

        try:
            # Filter for event type
            type_data = data.filter(pl.col("type") == event_type)
            log(f"      {event_type} data: {len(type_data):,} samples")

            if len(type_data) == 0:
                raise ValueError(f"No data found for event type: {event_type}")

            # Progressive sampling for large datasets
            if sample_size and len(type_data) > sample_size:
                # Stratified sampling to maintain label distribution
                positive_data = type_data.filter(pl.col("label") == 1)
                negative_data = type_data.filter(pl.col("label") == 0)

                pos_ratio = len(positive_data) / len(type_data)
                pos_sample_size = max(1, int(sample_size * pos_ratio))
                neg_sample_size = sample_size - pos_sample_size

                sampled_parts = []
                if len(positive_data) > 0:
                    if len(positive_data) > pos_sample_size:
                        positive_sampled = positive_data.sample(n=pos_sample_size, seed=42)
                    else:
                        positive_sampled = positive_data
                    sampled_parts.append(positive_sampled)

                if len(negative_data) > 0:
                    if len(negative_data) > neg_sample_size:
                        negative_sampled = negative_data.sample(n=neg_sample_size, seed=42)
                    else:
                        negative_sampled = negative_data
                    sampled_parts.append(negative_sampled)

                if sampled_parts:
                    type_data = pl.concat(sampled_parts)
                    log(f"      Sampled to: {len(type_data):,} samples")

            # Extract features and labels
            available_features = [col for col in self.feature_columns if col in type_data.columns]
            if len(available_features) == 0:
                raise ValueError(f"No feature columns found in data for {event_type}")

            log(f"      Using {len(available_features)} features")

            # Convert to numpy with fixed conversion method
            log(f"      Converting features to NumPy...")
            X = self.safe_polars_to_numpy(type_data, available_features)

            log(f"      Converting labels to NumPy...")
            y = self.safe_polars_to_numpy(type_data, ["label"]).flatten()

            # Convert to memory-efficient dtypes
            X = X.astype(config.FLOAT_DTYPE)
            y = y.astype(config.INT_DTYPE)

            # Store preprocessing stats
            self.preprocessing_stats[event_type] = {
                "original_samples": len(data.filter(pl.col("type") == event_type)),
                "final_samples": len(type_data),
                "features_used": len(available_features),
                "positive_rate": float(y.mean()),
                "feature_list": available_features
            }

            log(f"      Final shape: X={X.shape}, y={y.shape}, Positive rate: {y.mean():.4f}")

            # Cleanup
            del type_data
            force_garbage_collection()

            return X, y, available_features

        except Exception as e:
            log(f"      Error preparing {event_type} data: {str(e)}", "ERROR")
            raise e

## ENHANCED MODEL TRAINING

In [7]:
class FixedModelTrainer:
    """Fixed LightGBM trainer with robust GPU detection and fallbacks"""

    def __init__(self, use_gpu: bool = True):
        self.use_gpu = use_gpu and self._check_gpu_availability()
        self.models = {}
        self.training_results = {}
        self.feature_importance = {}

        log(f"Model trainer initialized - GPU: {'ENABLED' if self.use_gpu else 'DISABLED'}")

    def _check_gpu_availability(self) -> bool:
        """Robustly check if GPU is available for LightGBM"""
        try:
            # Check if LightGBM was compiled with GPU support
            import lightgbm as lgb

            # Create minimal test data
            X_test = np.random.random((50, 3)).astype(np.float32)
            y_test = np.random.randint(0, 2, 50)
            train_data = lgb.Dataset(X_test, label=y_test)

            # Try GPU training
            params = {
                'objective': 'binary',
                'device': 'gpu',
                'gpu_device_id': config.GPU_DEVICE_ID,
                'verbose': -1
            }

            model = lgb.train(params, train_data, num_boost_round=1, verbose_eval=False)
            log("    GPU availability test: PASSED")
            return True

        except Exception as e:
            log(f"    GPU availability test: FAILED ({str(e)})")
            return False

    def get_optimized_params(self, event_type: str, n_samples: int) -> Dict:
        """Get optimized parameters based on event type and data size"""

        # Base parameters for ranking
        params = {
            'objective': 'binary',  # Changed from lambdarank to binary for stability
            'metric': 'binary_logloss',
            'boosting_type': 'gbdt',
            'num_leaves': config.NUM_LEAVES,
            'learning_rate': config.LEARNING_RATE,
            'max_depth': config.MAX_DEPTH,
            'min_child_samples': config.MIN_CHILD_SAMPLES,
            'subsample': config.SUBSAMPLE,
            'colsample_bytree': config.COLSAMPLE_BYTREE,
            'reg_alpha': config.REG_ALPHA,
            'reg_lambda': config.REG_LAMBDA,
            'random_state': 42,
            'verbose': -1,
            'force_row_wise': True,
            'is_unbalance': True  # Handle imbalanced data
        }

        # GPU configuration
        if self.use_gpu:
            params.update({
                'device': 'gpu',
                'gpu_device_id': config.GPU_DEVICE_ID
            })
        else:
            params['device'] = 'cpu'

        # Event-specific adjustments
        if event_type == "clicks":
            # Clicks are more frequent, can handle more complexity
            params['num_leaves'] = min(127, int(config.NUM_LEAVES * 1.5))
            params['learning_rate'] = config.LEARNING_RATE * 0.9
        elif event_type == "orders":
            # Orders are sparse, need more regularization
            params['reg_alpha'] = config.REG_ALPHA * 2
            params['reg_lambda'] = config.REG_LAMBDA * 2
            params['min_child_samples'] = config.MIN_CHILD_SAMPLES * 2

        # Adaptive parameters based on data size
        if n_samples < 5000:
            params['num_leaves'] = max(15, params['num_leaves'] // 3)
            params['min_child_samples'] = max(5, params['min_child_samples'] // 2)
        elif n_samples < 20000:
            params['num_leaves'] = max(31, params['num_leaves'] // 2)
            params['min_child_samples'] = max(10, params['min_child_samples'] // 2)

        return params

    def train_model(self, X: np.ndarray, y: np.ndarray, event_type: str,
                   feature_names: List[str]) -> Optional[lgb.Booster]:
        """Train a robust LightGBM model with comprehensive error handling"""
        log(f"    Training {event_type} model...")

        try:
            start_time = datetime.now()

            # Validate input data
            if len(X) == 0 or len(y) == 0:
                raise ValueError(f"Empty training data for {event_type}")

            if X.shape[0] != y.shape[0]:
                raise ValueError(f"Mismatched X and y shapes: {X.shape[0]} vs {y.shape[0]}")

            # Check for valid labels
            unique_labels = np.unique(y)
            if len(unique_labels) < 2:
                log(f"      Warning: Only {len(unique_labels)} unique labels found", "WARN")
                if len(unique_labels) == 1:
                    # Add a few synthetic samples with the other label
                    if unique_labels[0] == 0:
                        y = np.append(y, [1] * min(10, len(y) // 10))
                        X = np.vstack([X, X[:min(10, len(y) // 10)]])
                    else:
                        y = np.append(y, [0] * min(10, len(y) // 10))
                        X = np.vstack([X, X[:min(10, len(y) // 10)]])

            # Split data with stratification
            try:
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, test_size=0.2, random_state=42, stratify=y
                )
            except ValueError:
                # Fallback without stratification if it fails
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, test_size=0.2, random_state=42
                )

            log(f"      Train: {X_train.shape[0]:,}, Val: {X_val.shape[0]:,}")
            log(f"      Train labels: {np.bincount(y_train)}, Val labels: {np.bincount(y_val)}")

            # Create LightGBM datasets with error handling
            try:
                train_data = lgb.Dataset(
                    X_train, label=y_train,
                    feature_name=feature_names,
                    free_raw_data=False
                )

                val_data = lgb.Dataset(
                    X_val, label=y_val,
                    feature_name=feature_names,
                    reference=train_data,
                    free_raw_data=False
                )
            except Exception as e:
                log(f"      Error creating LightGBM datasets: {e}", "ERROR")
                raise e

            # Get optimized parameters
            params = self.get_optimized_params(event_type, len(X))
            log(f"      Using {'GPU' if params.get('device') == 'gpu' else 'CPU'} device")

            # Training with early stopping and comprehensive callbacks
            callbacks = [
                lgb.early_stopping(config.EARLY_STOPPING_ROUNDS, verbose=False),
                lgb.log_evaluation(period=0)  # Silent training
            ]

            # Train with error handling for GPU fallback
            try:
                model = lgb.train(
                    params,
                    train_data,
                    valid_sets=[train_data, val_data],
                    valid_names=['train', 'valid'],
                    num_boost_round=config.N_ESTIMATORS,
                    callbacks=callbacks
                )
            except Exception as gpu_error:
                if self.use_gpu and 'gpu' in str(gpu_error).lower():
                    log(f"      GPU training failed, falling back to CPU: {gpu_error}", "WARN")
                    # Fallback to CPU
                    params['device'] = 'cpu'
                    if 'gpu_device_id' in params:
                        del params['gpu_device_id']

                    model = lgb.train(
                        params,
                        train_data,
                        valid_sets=[train_data, val_data],
                        valid_names=['train', 'valid'],
                        num_boost_round=config.N_ESTIMATORS,
                        callbacks=callbacks
                    )
                else:
                    raise gpu_error

            training_time = (datetime.now() - start_time).total_seconds()

            # Get best score (handle different score formats)
            best_score = 0.0
            if hasattr(model, 'best_score') and model.best_score:
                valid_scores = model.best_score.get('valid', {})
                best_score = valid_scores.get('binary_logloss', 0.0)

            # Store results
            self.training_results[event_type] = {
                "training_samples": len(X_train),
                "validation_samples": len(X_val),
                "features_used": len(feature_names),
                "best_iteration": getattr(model, 'best_iteration', config.N_ESTIMATORS),
                "best_score": best_score,
                "training_time": training_time,
                "params_used": params,
                "positive_rate": float(y.mean())
            }

            # Store feature importance
            try:
                importance = model.feature_importance(importance_type='gain')
                self.feature_importance[event_type] = {
                    feature_names[i]: float(importance[i])
                    for i in range(len(feature_names))
                }
            except Exception as e:
                log(f"      Warning: Could not extract feature importance: {e}", "WARN")
                self.feature_importance[event_type] = {}

            log(f"      Training completed: {training_time:.1f}s, Score: {best_score:.4f}")

            # Store model
            self.models[event_type] = model

            # Cleanup
            del X_train, X_val, y_train, y_val, train_data, val_data
            force_garbage_collection()

            return model

        except Exception as e:
            log(f"      Training failed for {event_type}: {str(e)}", "ERROR")
            self.training_results[event_type] = {"error": str(e)}
            return None

## ENHANCED MODEL EVALUATION

In [8]:
class FixedModelEvaluator:
    """Enhanced model evaluation with robust metrics calculation"""

    def __init__(self):
        self.evaluation_results = {}

    def evaluate_model(self, model: lgb.Booster, X_test: np.ndarray,
                      y_test: np.ndarray, event_type: str) -> Dict:
        """Comprehensive model evaluation with robust error handling"""
        log(f"    Evaluating {event_type} model...")

        try:
            # Predictions with error handling
            try:
                y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            except:
                y_pred = model.predict(X_test)

            # Basic validation
            if len(y_pred) != len(y_test):
                raise ValueError(f"Prediction length mismatch: {len(y_pred)} vs {len(y_test)}")

            # Calculate metrics
            metrics = {}

            # Basic prediction statistics
            metrics.update({
                'mean_prediction': float(np.mean(y_pred)),
                'std_prediction': float(np.std(y_pred)),
                'min_prediction': float(np.min(y_pred)),
                'max_prediction': float(np.max(y_pred)),
                'actual_positive_rate': float(np.mean(y_test)),
                'test_samples': len(y_test)
            })

            # Calculate recall and precision metrics
            if len(y_test) > 0 and np.sum(y_test) > 0:
                # Sort by prediction scores
                sorted_indices = np.argsort(y_pred)[::-1]
                sorted_labels = y_test[sorted_indices]
                total_positives = np.sum(y_test)

                # Calculate recall at different K values
                for k in config.EVAL_AT_K:
                    if k <= len(sorted_labels):
                        top_k_labels = sorted_labels[:k]
                        recall_k = np.sum(top_k_labels) / max(1, total_positives)
                        precision_k = np.sum(top_k_labels) / max(1, k)
                        metrics[f'recall_at_{k}'] = float(recall_k)
                        metrics[f'precision_at_{k}'] = float(precision_k)
                    else:
                        metrics[f'recall_at_{k}'] = 0.0
                        metrics[f'precision_at_{k}'] = 0.0

                # Calculate NDCG with error handling
                for k in config.EVAL_AT_K:
                    try:
                        if len(np.unique(y_test)) > 1 and k <= len(y_test):
                            ndcg_k = ndcg_score([y_test], [y_pred], k=k)
                            metrics[f'ndcg_at_{k}'] = float(ndcg_k)
                        else:
                            metrics[f'ndcg_at_{k}'] = 0.0
                    except Exception as e:
                        log(f"        NDCG@{k} calculation failed: {e}", "WARN")
                        metrics[f'ndcg_at_{k}'] = 0.0
            else:
                # No positive samples
                for k in config.EVAL_AT_K:
                    metrics[f'recall_at_{k}'] = 0.0
                    metrics[f'precision_at_{k}'] = 0.0
                    metrics[f'ndcg_at_{k}'] = 0.0

            self.evaluation_results[event_type] = metrics

            log(f"      Evaluation completed - NDCG@20: {metrics.get('ndcg_at_20', 0):.4f}, "
                f"Recall@20: {metrics.get('recall_at_20', 0):.4f}")

            return metrics

        except Exception as e:
            log(f"      Evaluation failed for {event_type}: {str(e)}", "ERROR")
            error_metrics = {f'{metric}_at_{k}': 0.0 for metric in ['ndcg', 'recall', 'precision'] for k in config.EVAL_AT_K}
            error_metrics.update({
                'error': str(e),
                'test_samples': len(y_test) if 'y_test' in locals() else 0
            })
            self.evaluation_results[event_type] = error_metrics
            return error_metrics

    def calculate_weighted_metrics(self) -> Dict:
        """Calculate weighted metrics across all event types"""

        # Event type weights (based on business importance)
        weights = {"clicks": 0.10, "carts": 0.30, "orders": 0.60}

        weighted_metrics = {}

        for metric in ['ndcg_at_20', 'recall_at_20', 'precision_at_20']:
            weighted_sum = 0.0
            total_weight = 0.0

            for event_type, weight in weights.items():
                if event_type in self.evaluation_results:
                    metric_value = self.evaluation_results[event_type].get(metric, 0.0)
                    if not np.isnan(metric_value):
                        weighted_sum += metric_value * weight
                        total_weight += weight

            if total_weight > 0:
                weighted_metrics[f'weighted_{metric}'] = weighted_sum / total_weight
            else:
                weighted_metrics[f'weighted_{metric}'] = 0.0

        return weighted_metrics

## MAIN TRAINING EXECUTION

In [9]:
def run_fixed_training():
    """Execute fixed training pipeline with comprehensive error handling"""
    log("Starting fixed training pipeline...")

    # Initialize components
    preprocessor = FixedDataPreprocessor(feature_columns)
    trainer = FixedModelTrainer(use_gpu=config.USE_GPU)
    evaluator = FixedModelEvaluator()

    trained_models = {}

    # Train models for each event type
    for event_type in ["clicks", "carts", "orders"]:
        log(f"  Processing {event_type} model...")

        try:
            # Prepare training data
            X_train, y_train, used_features = preprocessor.prepare_training_data(
                val_data, event_type, config.TRAINING_SAMPLE_SIZE
            )

            # Validate prepared data
            if len(X_train) < 100:
                log(f"      Insufficient training data for {event_type}: {len(X_train)} samples", "WARN")
                continue

            # Train model
            model = trainer.train_model(X_train, y_train, event_type, used_features)

            if model is not None:
                trained_models[event_type] = model

                # Evaluate model
                try:
                    X_eval, y_eval, _ = preprocessor.prepare_training_data(
                        val_data, event_type, config.VALIDATION_SAMPLE_SIZE
                    )

                    if len(X_eval) > 0:
                        evaluator.evaluate_model(model, X_eval, y_eval, event_type)

                    # Cleanup evaluation data
                    del X_eval, y_eval
                except Exception as eval_error:
                    log(f"      Evaluation failed for {event_type}: {eval_error}", "WARN")

            # Cleanup training data
            del X_train, y_train
            force_garbage_collection()

            log(f"    {event_type} processing completed [Memory: {get_memory_usage():.1f}GB]")

        except Exception as e:
            log(f"    {event_type} processing failed: {str(e)}", "ERROR")
            continue

    # Calculate weighted metrics
    weighted_metrics = evaluator.calculate_weighted_metrics()

    return trained_models, trainer.training_results, evaluator.evaluation_results, weighted_metrics, trainer.feature_importance

# Execute training
trained_models, training_results, evaluation_results, weighted_metrics, feature_importance = run_fixed_training()

[2025-08-08 10:26:51] [INFO] [MEM: 4.8GB/12.6%/OK] Starting fixed training pipeline...
[2025-08-08 10:26:51] [INFO] [MEM: 4.8GB/12.6%/OK]     GPU availability test: FAILED (train() got an unexpected keyword argument 'verbose_eval')
[2025-08-08 10:26:51] [INFO] [MEM: 4.8GB/12.6%/OK] Model trainer initialized - GPU: DISABLED
[2025-08-08 10:26:51] [INFO] [MEM: 4.8GB/12.6%/OK]   Processing clicks model...
[2025-08-08 10:26:51] [INFO] [MEM: 4.8GB/12.6%/OK]     Preparing clicks training data...
[2025-08-08 10:26:51] [INFO] [MEM: 6.0GB/15.6%/OK]       clicks data: 2,313,996 samples
[2025-08-08 10:26:51] [INFO] [MEM: 7.2GB/18.8%/OK]       Sampled to: 100,000 samples
[2025-08-08 10:26:51] [INFO] [MEM: 7.2GB/18.8%/OK]       Using 61 features
[2025-08-08 10:26:51] [INFO] [MEM: 7.2GB/18.8%/OK]       Converting features to NumPy...
[2025-08-08 10:26:51] [INFO] [MEM: 7.2GB/18.8%/OK]       Attempting direct Polars to NumPy conversion...
[2025-08-08 10:26:51] [INFO] [MEM: 7.2GB/18.9%/OK]       Convert

## ENHANCED OUTPUT SAVING

In [10]:
def save_fixed_outputs(models: Dict, training_results: Dict, evaluation_results: Dict,
                      weighted_metrics: Dict, feature_importance: Dict) -> Dict:
    """Save all outputs with enhanced error handling and validation"""
    log("Saving fixed outputs...")

    saved_files = {}

    try:
        # 1. Save trained models with validation
        for event_type, model in models.items():
            try:
                model_path = f"{config.OUTPUT_PATH}/ranker_{event_type}.txt"
                model.save_model(model_path)

                # Validate the saved model
                if os.path.exists(model_path):
                    file_size = os.path.getsize(model_path) / (1024*1024)
                    saved_files[f"ranker_{event_type}"] = model_path
                    log(f"    ranker_{event_type}.txt saved ({file_size:.1f} MB)")

                    # Test loading the saved model
                    try:
                        test_model = lgb.Booster(model_file=model_path)
                        log(f"    ranker_{event_type}.txt validation: PASSED")
                    except Exception as load_error:
                        log(f"    ranker_{event_type}.txt validation: FAILED - {load_error}", "ERROR")
                else:
                    log(f"    Error: ranker_{event_type}.txt was not created", "ERROR")

            except Exception as e:
                log(f"    Error saving {event_type} model: {e}", "ERROR")

        # 2. Save training results (use pickle to avoid JSON serialization issues)
        try:
            training_path = f"{config.OUTPUT_PATH}/model_training_results.pkl"
            with open(training_path, "wb") as f:
                pickle.dump(training_results, f)
            saved_files["training_results"] = training_path
            log(f"    model_training_results.pkl saved")
        except Exception as e:
            log(f"    Error saving training results: {e}", "ERROR")

        # 3. Save evaluation results (BOTH pickle and JSON formats)
        try:
            # Save as pickle (detailed format)
            evaluation_path = f"{config.OUTPUT_PATH}/model_evaluation_results.pkl"
            combined_evaluation = {
                "individual_metrics": evaluation_results,
                "weighted_metrics": weighted_metrics,
                "evaluation_timestamp": datetime.now().isoformat(),
                "config_used": {
                    "training_sample_size": config.TRAINING_SAMPLE_SIZE,
                    "validation_sample_size": config.VALIDATION_SAMPLE_SIZE,
                    "n_estimators": config.N_ESTIMATORS,
                    "num_leaves": config.NUM_LEAVES,
                    "learning_rate": config.LEARNING_RATE
                }
            }
            with open(evaluation_path, "wb") as f:
                pickle.dump(combined_evaluation, f)
            saved_files["evaluation_results"] = evaluation_path
            log(f"    model_evaluation_results.pkl saved")

            # CRITICAL FIX: Save as JSON for Part 3 compatibility
            json_evaluation_path = f"{config.OUTPUT_PATH}/evaluation_results.json"

            # Create the JSON structure that Part 3 expects
            json_evaluation = {
                # Extract individual event type metrics (NDCG@20 as primary metric)
                "clicks": evaluation_results.get("clicks", {}).get("ndcg_at_20", 0.0),
                "carts": evaluation_results.get("carts", {}).get("ndcg_at_20", 0.0),
                "orders": evaluation_results.get("orders", {}).get("ndcg_at_20", 0.0),

                # Weighted average (use NDCG@20 as primary metric)
                "weighted_average": weighted_metrics.get("weighted_ndcg_at_20", 0.0),

                # Additional metrics for comprehensive reporting
                "detailed_metrics": {
                    "weighted_recall_at_20": weighted_metrics.get("weighted_recall_at_20", 0.0),
                    "weighted_precision_at_20": weighted_metrics.get("weighted_precision_at_20", 0.0),
                    "weighted_ndcg_at_20": weighted_metrics.get("weighted_ndcg_at_20", 0.0)
                },

                # Individual event type detailed metrics
                "individual_metrics": {
                    event_type: {
                        "ndcg_at_20": metrics.get("ndcg_at_20", 0.0),
                        "recall_at_20": metrics.get("recall_at_20", 0.0),
                        "precision_at_20": metrics.get("precision_at_20", 0.0),
                        "test_samples": metrics.get("test_samples", 0)
                    }
                    for event_type, metrics in evaluation_results.items()
                    if isinstance(metrics, dict) and "error" not in metrics
                },

                # Metadata
                "evaluation_timestamp": datetime.now().isoformat(),
                "models_evaluated": list(models.keys()),
                "total_models": len(models)
            }

            # Save JSON file with error handling
            with open(json_evaluation_path, "w") as f:
                json.dump(json_evaluation, f, indent=2)
            saved_files["evaluation_results_json"] = json_evaluation_path
            log(f"    evaluation_results.json saved")

        except Exception as e:
            log(f"    Error saving evaluation results: {e}", "ERROR")

        # 4. Save enhanced feature importance
        try:
            importance_path = f"{config.OUTPUT_PATH}/enhanced_feature_importance.pkl"
            enhanced_importance = {
                "individual_importance": feature_importance,
                "aggregated_importance": {},
                "top_features": {},
                "feature_ranking": {}
            }

            # Calculate aggregated importance across all models
            all_features = set()
            for event_importance in feature_importance.values():
                all_features.update(event_importance.keys())

            for feature in all_features:
                total_importance = sum(
                    event_importance.get(feature, 0)
                    for event_importance in feature_importance.values()
                )
                enhanced_importance["aggregated_importance"][feature] = total_importance

            # Get top features
            if enhanced_importance["aggregated_importance"]:
                sorted_features = sorted(
                    enhanced_importance["aggregated_importance"].items(),
                    key=lambda x: x[1], reverse=True
                )
                enhanced_importance["top_features"] = dict(sorted_features[:20])
                enhanced_importance["feature_ranking"] = {
                    feature: rank + 1 for rank, (feature, _) in enumerate(sorted_features)
                }

            with open(importance_path, "wb") as f:
                pickle.dump(enhanced_importance, f)
            saved_files["feature_importance"] = importance_path
            log(f"    enhanced_feature_importance.pkl saved")
        except Exception as e:
            log(f"    Error saving feature importance: {e}", "ERROR")

        # 5. Save comprehensive summary
        try:
            summary = {
                "notebook": "Part 2B4: FIXED Model Training & Evaluation",
                "completion_timestamp": datetime.now().isoformat(),
                "version": "FIXED - Robust Polars conversion and comprehensive error handling",
                "critical_fixes_applied": [
                    "Fixed Polars to_numpy() conversion with multiple fallback methods",
                    "Robust GPU detection with automatic CPU fallback",
                    "Enhanced data validation and preprocessing",
                    "Improved error handling throughout the pipeline",
                    "Binary classification instead of lambdarank for stability",
                    "Comprehensive model validation after saving",
                    "Memory-optimized data processing",
                    "Stratified sampling with fallbacks",
                    "CRITICAL: Added JSON evaluation results for Part 3 compatibility"
                ],
                "inputs_used": {
                    "val_data_features.parquet": f"{validation_results['data_shapes']['val_data'][0]:,} samples",
                    "feature_columns": f"{len(feature_columns)} features",
                    "memory_configuration": f"{config.MAX_MEMORY_GB:.1f}GB max"
                },
                "outputs_generated": {
                    "trained_models": list(models.keys()),
                    "model_files": [f"ranker_{et}.txt" for et in models.keys()],
                    "training_results": "Comprehensive training metrics",
                    "evaluation_results": "Multi-metric evaluation with NDCG and Recall",
                    "evaluation_results_json": "JSON format for Part 3 compatibility",
                    "feature_importance": "Enhanced feature analysis and ranking"
                },
                "performance_metrics": {
                    "models_trained": len(models),
                    "successful_models": [et for et in models.keys()],
                    "failed_models": [et for et in ["clicks", "carts", "orders"] if et not in models],
                    "weighted_recall_at_20": weighted_metrics.get("weighted_recall_at_20", 0),
                    "weighted_ndcg_at_20": weighted_metrics.get("weighted_ndcg_at_20", 0),
                    "weighted_precision_at_20": weighted_metrics.get("weighted_precision_at_20", 0),
                    "gpu_used": any(tr.get("params_used", {}).get("device") == "gpu" for tr in training_results.values()),
                    "final_memory_usage_gb": get_memory_usage(),
                    "training_successful": len(models) >= 1
                },
                "quality_assessment": {
                    "models_trained_successfully": len(models) >= 1,
                    "at_least_two_models": len(models) >= 2,
                    "performance_acceptable": weighted_metrics.get("weighted_recall_at_20", 0) > config.MIN_RECALL_THRESHOLD,
                    "memory_efficient": get_memory_usage() < config.MEMORY_THRESHOLD,
                    "ready_for_inference": len(models) >= 1 and all(os.path.exists(f"{config.OUTPUT_PATH}/ranker_{et}.txt") for et in models.keys()),
                    "json_compatibility": os.path.exists(f"{config.OUTPUT_PATH}/evaluation_results.json")
                },
                "next_step": "Ready for Part 3: Inference & Submission Generation" if len(models) >= 1 else "Review errors and retry training"
            }

            summary_path = f"{config.OUTPUT_PATH}/part_2b4_fixed_summary.pkl"
            with open(summary_path, "wb") as f:
                pickle.dump(summary, f)
            saved_files["summary"] = summary_path
            log(f"    part_2b4_fixed_summary.pkl saved")

        except Exception as e:
            log(f"    Error saving summary: {e}", "ERROR")

        log(f"Fixed outputs saved successfully!")
        return saved_files

    except Exception as e:
        log(f"Error in save_fixed_outputs: {e}", "ERROR")
        return saved_files

# Save all outputs
saved_files = save_fixed_outputs(trained_models, training_results, evaluation_results, weighted_metrics, feature_importance)

[2025-08-08 10:27:01] [INFO] [MEM: 8.1GB/21.1%/OK] Saving fixed outputs...
[2025-08-08 10:27:01] [INFO] [MEM: 8.1GB/21.1%/OK]     ranker_clicks.txt saved (0.0 MB)
[2025-08-08 10:27:01] [INFO] [MEM: 8.1GB/21.1%/OK]     ranker_clicks.txt validation: PASSED
[2025-08-08 10:27:01] [INFO] [MEM: 8.1GB/21.1%/OK]     ranker_carts.txt saved (0.0 MB)
[2025-08-08 10:27:01] [INFO] [MEM: 8.1GB/21.1%/OK]     ranker_carts.txt validation: PASSED
[2025-08-08 10:27:01] [INFO] [MEM: 8.1GB/21.1%/OK]     ranker_orders.txt saved (0.0 MB)
[2025-08-08 10:27:02] [INFO] [MEM: 8.1GB/21.1%/OK]     ranker_orders.txt validation: PASSED
[2025-08-08 10:27:02] [INFO] [MEM: 8.1GB/21.1%/OK]     model_training_results.pkl saved
[2025-08-08 10:27:02] [INFO] [MEM: 8.1GB/21.1%/OK]     model_evaluation_results.pkl saved
[2025-08-08 10:27:02] [INFO] [MEM: 8.1GB/21.1%/OK]     evaluation_results.json saved
[2025-08-08 10:27:02] [INFO] [MEM: 8.1GB/21.1%/OK]     enhanced_feature_importance.pkl saved
[2025-08-08 10:27:03] [INFO] [M

## ENHANCED FINAL SUMMARY

In [11]:
log("\n" + "="*80)
log("PART 2B4 FIXED TRAINING COMPLETED")
log("="*80)

# Display training results
log(f"\nTRAINING RESULTS:")
log(f"  Models successfully trained: {len(trained_models)}/3")
for event_type in ["clicks", "carts", "orders"]:
    if event_type in training_results:
        results = training_results[event_type]
        if "error" not in results:
            samples = results.get("training_samples", 0)
            best_score = results.get("best_score", 0)
            training_time = results.get("training_time", 0)
            device = results.get("params_used", {}).get("device", "unknown")
            log(f"  {event_type}: SUCCESS - {samples:,} samples, Score={best_score:.4f}, {training_time:.1f}s ({device})")
        else:
            log(f"  {event_type}: FAILED - {results['error']}")
    else:
        log(f"  {event_type}: NOT ATTEMPTED")

# Display evaluation results
if evaluation_results:
    log(f"\nEVALUATION RESULTS:")
    for event_type in ["clicks", "carts", "orders"]:
        if event_type in evaluation_results:
            metrics = evaluation_results[event_type]
            if "error" not in metrics:
                ndcg = metrics.get("ndcg_at_20", 0)
                recall = metrics.get("recall_at_20", 0)
                precision = metrics.get("precision_at_20", 0)
                samples = metrics.get("test_samples", 0)
                log(f"  {event_type}: NDCG@20={ndcg:.4f}, Recall@20={recall:.4f}, Precision@20={precision:.4f} ({samples:,} samples)")
            else:
                log(f"  {event_type}: EVALUATION FAILED - {metrics['error']}")
        else:
            log(f"  {event_type}: NOT EVALUATED")

# Display weighted metrics
if weighted_metrics:
    log(f"\nWEIGHTED PERFORMANCE:")
    log(f"  Weighted NDCG@20: {weighted_metrics.get('weighted_ndcg_at_20', 0):.4f}")
    log(f"  Weighted Recall@20: {weighted_metrics.get('weighted_recall_at_20', 0):.4f}")
    log(f"  Weighted Precision@20: {weighted_metrics.get('weighted_precision_at_20', 0):.4f}")

# Display top features
if feature_importance:
    log(f"\nTOP FEATURES (aggregated importance):")
    all_features = {}
    for event_importance in feature_importance.values():
        for feature, importance in event_importance.items():
            all_features[feature] = all_features.get(feature, 0) + importance

    if all_features:
        top_features = sorted(all_features.items(), key=lambda x: x[1], reverse=True)[:10]
        for i, (feature, importance) in enumerate(top_features):
            log(f"  {i+1:2d}. {feature}: {importance:.2f}")
    else:
        log(f"  No feature importance data available")

# Display output files
log(f"\nOUTPUT FILES GENERATED:")
for file_type, file_path in saved_files.items():
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / (1024*1024)
        log(f"  {os.path.basename(file_path)} ({file_size:.1f} MB)")
    else:
        log(f"  {os.path.basename(file_path)} (FILE NOT FOUND)", "ERROR")
log(f"  All files saved to: {config.OUTPUT_PATH}")

# Model file validation
log(f"\nMODEL FILE VALIDATION:")
for event_type in ["clicks", "carts", "orders"]:
    model_path = f"{config.OUTPUT_PATH}/ranker_{event_type}.txt"
    if os.path.exists(model_path):
        try:
            test_model = lgb.Booster(model_file=model_path)
            log(f"  ranker_{event_type}.txt: VALID ✓")
        except Exception as e:
            log(f"  ranker_{event_type}.txt: INVALID ✗ ({e})", "ERROR")
    else:
        log(f"  ranker_{event_type}.txt: NOT FOUND ✗")

# Quality assessment
models_ok = len(trained_models) >= 1
performance_ok = weighted_metrics.get("weighted_recall_at_20", 0) > config.MIN_RECALL_THRESHOLD
memory_ok = get_memory_usage() < config.MEMORY_THRESHOLD
files_ok = all(os.path.exists(f"{config.OUTPUT_PATH}/ranker_{et}.txt") for et in trained_models.keys())

log(f"\nQUALITY ASSESSMENT:")
log(f"  Models trained (≥1): {'PASS' if models_ok else 'FAIL'} ({len(trained_models)}/3)")
log(f"  Performance (>{config.MIN_RECALL_THRESHOLD:.3f}): {'PASS' if performance_ok else 'FAIL'} ({weighted_metrics.get('weighted_recall_at_20', 0):.4f})")
log(f"  Memory efficiency: {'PASS' if memory_ok else 'FAIL'} ({get_memory_usage():.1f}GB)")
log(f"  Model files created: {'PASS' if files_ok else 'FAIL'}")
log(f"  Ready for inference: {'YES' if models_ok and files_ok else 'NO'}")

overall_quality = "EXCELLENT" if models_ok and performance_ok and memory_ok and files_ok else "GOOD" if models_ok and files_ok else "NEEDS_IMPROVEMENT"
log(f"\nOverall Quality: {overall_quality}")

# Performance summary
log(f"\nPERFORMANCE SUMMARY:")
log(f"  Final memory usage: {get_memory_usage():.2f} GB")
log(f"  Memory threshold: {config.MEMORY_THRESHOLD:.1f} GB")
log(f"  GPU acceleration attempted: {config.USE_GPU}")
log(f"  Training optimization: FIXED AND ROBUST")

# Final cleanup
log(f"\nPerforming final cleanup...")
try:
    force_garbage_collection()
    final_memory = get_memory_usage()
    log(f"Final memory after cleanup: {final_memory:.2f} GB")
except Exception as e:
    log(f"Cleanup warning: {e}", "WARN")

# Final status
if len(trained_models) >= 1 and files_ok:
    log(f"\n SUCCESS: Part 2B4 Fixed Training completed successfully!")
    log(f" {len(trained_models)} models trained and saved")
    log(f" Ready for Part 3: Inference & Submission Generation")
else:
    log(f"\n  PARTIAL SUCCESS: Some issues remain")
    log(f" {len(trained_models)} out of 3 models trained")
    log(f" Review errors above and consider re-running")

log("="*80)

[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK] 
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK] PART 2B4 FIXED TRAINING COMPLETED
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK] ================================================================================
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK] 
TRAINING RESULTS:
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK]   Models successfully trained: 3/3
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK]   clicks: SUCCESS - 80,000 samples, Score=0.1859, 0.5s (cpu)
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK]   carts: SUCCESS - 80,000 samples, Score=0.2979, 1.0s (cpu)
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK]   orders: SUCCESS - 80,000 samples, Score=0.3441, 0.5s (cpu)
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK] 
EVALUATION RESULTS:
[2025-08-08 10:27:03] [INFO] [MEM: 8.1GB/21.1%/OK]   clicks: NDCG@20=0.2992, Recall@20=0.0047, Precision@20=0.2500 (50,000 samples)
[2025-08-08 10:27:03] [INFO] [MEM: 8